# Homework 1: Introduction for [LLM Zoomcamp 2024](https://courses.datatalks.club/llm-zoomcamp-2024/)
- [homework](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md)
- Due date: 2 July 2024 01:00 (local time)
- [Submit here](https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1)
    - use private github account for log in

## Extra questions

- Homework URL
- Time spent on lectures (hours): ~4h
- Time spent on homework (hours): ~1h

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

### Answer

`42f05b9372a9a4a470db3b52817899b99a76ee73`

### Code

In terminal: Running ElasticSearch:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    -e "ES_JAVA_OPTS=-Xms512m -Xmx512m" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

`-e "ES_JAVA_OPTS=-Xms512m -Xmx512m" ` added: Elasticsearch might be trying to allocate more memory than is available in your Codespace. limit the memory Elasticsearch tries to allocate by setting the ES_JAVA_OPTS environment variable to allocate less memory.

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch("http://localhost:9200")

In [3]:
es_client.info()["version"]["build_hash"]
# '42f05b9372a9a4a470db3b52817899b99a76ee73'

'42f05b9372a9a4a470db3b52817899b99a76ee73'

## Getting the data

In [4]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [5]:
# test
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

### Answer

`index`

### Code

In [7]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)

In [9]:
from tqdm.auto import tqdm

In [11]:
for doc in tqdm(documents):
    # es_client.index(index=index_name, document=doc)
    pass

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

## Answer

`84.050095`

### Code

In [12]:
boost = 4
fields = [f"question^{boost}", "text"]
type = "best_fields"
size = 1


def elastic_search_q3(query):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": type,
                    }
                },
            }
        },
    }
    return es_client.search(index=index_name, body=search_query)

In [13]:
query = "How do I execute a command in a running docker container?"
response = elastic_search_q3(query)

In [14]:
response["hits"]["hits"][0]["_score"]

84.050095

In [15]:
response["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'utCpS5ABMddOhdJdOIla',
 '_score': 84.050095,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

### Answer

`'How do I copy files from a different folder into docker container’s working directory?'`

### Code

In [16]:
boost = 4
fields = [f"question^{boost}", "text"]
type = "best_fields"
size = 3
filter = {"term": {"course": "machine-learning-zoomcamp"}}


def elastic_search_q4(query):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": type,
                    },
                },
                "filter": filter,
            },
        },
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"]["question"])

    return result_docs

In [17]:
query = "How do I execute a command in a running docker container?"
result_docs = elastic_search_q4(query)

In [18]:
result_docs[2]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

### Answer

`1462`

### Code

#### Search Results

In [19]:
boost = 4
fields = [f"question^{boost}", "text"]
type = "best_fields"
size = 3
filter = {"term": {"course": "machine-learning-zoomcamp"}}


def elastic_search_q5(query):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": type,
                    },
                },
                "filter": filter,
            },
        },
    }

    response = es_client.search(index=index_name, body=search_query)

    return [hit["_source"] for hit in response["hits"]["hits"]]

In [20]:
query = "How do I execute a command in a running docker container?"
search_results = elastic_search_q5(query)

In [21]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

#### Build Prompt

In [22]:
context_template = """
Q: {question}
A: {text}
""".strip()


def build_context(search_results):
    context = ""
    for doc in search_results:
        context += context_template.format(question=doc["question"], text=doc["text"]) + "\n\n"

    return context

In [23]:
print(build_context(search_results))

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [24]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


def build_prompt(query, search_results):
    context = build_context(search_results)
    return prompt_template.format(question=query, context=context).strip()

In [25]:
query = "How do I execute a command in a running docker container?"
search_results = elastic_search_q5(query)
prompt = build_prompt(query, search_results)

In [26]:
print(f"Question 5. Prompt length  {len(prompt)}")

Question 5. Prompt length  1462


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

### Answer

`322`

In [27]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [28]:
# Encode the text to get the tokens
tokens = encoding.encode(prompt)

In [29]:
print(f"Question 6. Number of tokens {len(tokens)}")

Question 6. Number of tokens 322


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [30]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

### Reply using ChatGPT 4o

In [40]:
ChatGPT_4o_reponse = """
To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps:

1. Find the container ID of the running container by using:
   ```bash
   docker ps
   ```

2. Once you have the container ID, execute a command in the container by using:
   ```bash
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session in the container:
   ```bash
   docker exec -it <container-id> bash
   ```
"""

In [41]:
print(ChatGPT_4o_reponse)


To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps:

1. Find the container ID of the running container by using:
   ```bash
   docker ps
   ```

2. Once you have the container ID, execute a command in the container by using:
   ```bash
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session in the container:
   ```bash
   docker exec -it <container-id> bash
   ```



### Reply using Grow

In [33]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

True

In [34]:
model_id = "llama3-70b-8192"
api_key = os.getenv("GROQ_TOKEN")

In [36]:
def get_answer_text(model_id, api_key, user_input, model_temp=0.7) -> tuple[str, dict]:
    response = Groq(api_key=api_key).chat.completions.create(
        model=model_id,
        messages=[{"role": "user", "content": user_input}],
        temperature=model_temp,
    )

    token_usage = dict(response.usage)
    content: str = str(response.choices[0].message.content).strip()
    return content, token_usage

In [37]:
content, token_usage = get_answer_text(model_id, api_key, prompt)

In [42]:
print(content)

According to the FAQ database, to execute a command in a running Docker container, you can use the following command:

`docker exec -it <container-id> bash`

First, find the container-id using `docker ps`, and then execute the command in the specific container using `docker exec`.


In [38]:
token_usage

{'completion_tokens': 60,
 'prompt_tokens': 334,
 'total_tokens': 394,
 'completion_time': 0.161827661,
 'prompt_time': 0.094205751,
 'queue_time': None,
 'total_time': 0.256033412}